quelle : https://www.govdata.de/suche/daten/donau-ries-lkr-apfel-und-birnen

In [200]:
import requests

url = "https://fms.donau-ries.de/formcycle/datenabfragedb?mandantName=LRA-Donau-Ries&name=aepfel-und-birnen"

# Anfrage schicken
response = requests.get(url)

# prüfen ob alles ok war
if response.status_code == 200:
    data = response.json()   # JSON direkt als Python-Dict/Liste
    #print(data['result'])
else:
    print("Fehler beim Abrufen:", response.status_code)



In [201]:
from bs4 import BeautifulSoup
ls_keys = ["geschichte", "frucht", "reifezeit", "standort", "anfälligkeit", "blüte", "sonstiges" ]
alle_sorten = {}
number  = 0
for eintrag in data["result"]:
    alias = eintrag["alias"]
    
    html_text = eintrag["text_inhalt"]
    #quellen = eintrag.get("quellen", "")

    soup = BeautifulSoup(html_text, "html.parser")

    sorten_dict = {}
    current_key = None
    #print(soup)
    for tag in soup.find_all("p"):
        text = tag.get_text(strip=True)
        #print(tag)
        # Neue Sektion wenn <strong> drin ist
        if tag.find("strong"):
            current_key = text.replace(":", "").lower()
            if current_key=="früchte":
                current_key="frucht"
            if current_key=="früchte":
                current_key="frucht"
            sorten_dict[current_key] = ""
        else:
            if current_key:
                if sorten_dict[current_key]:
                    sorten_dict[current_key] += " " + text
                else:
                    sorten_dict[current_key] = text
    sorten_dict["ind"] = number
    alle_sorten[alias] = sorten_dict
    number += 1
#print(alle_sorten["blauapfel"])
#alle_sorten["horneburger-pfannkuchenapfel"]

#alle_sorten["suesskirsche-skeena"]

In [202]:

LS_KEYS = ["geschichte", "frucht", "reifezeit", "standort", "anfälligkeit", "blüte", "sonstiges", "ind"]
alle_sorten_cleaned  = {}
sorten_cleaned  = {}
for key, value in alle_sorten.items():
    
    for ke, val in value.items():
        if ke in LS_KEYS:
            sorten_cleaned[ke]= val
            continue
        else:
            for wort in LS_KEYS:
                if ke.startswith(wort):
                    sorten_cleaned[wort] =  ke[len(wort)::] 
                else:
                    continue
    alle_sorten_cleaned[key] = sorten_cleaned
    sorten_cleaned  = {}



In [203]:
import pandas as pd
df = pd.DataFrame(alle_sorten_cleaned)
trans  = df.T
lss = trans[trans["anfälligkeit"].notna()]["anfälligkeit"].tolist()
lsn = trans[trans["anfälligkeit"].notna()]["ind"].tolist()



In [216]:
anf = dict(zip(lsn,lss))

In [222]:
mask = []
for i in anf.values():
    mask.append("feuer" in i.lower())
anfm = dict(zip(lsn,mask))
anf

{0: 'Feuerbrand, Mehltau, Spinnmilben, Triebsucht',
 1: 'Stark Feuerbrand gefährdet. Stark Mehltau anfällig, robust für Apfelschorf. Auf schweren oder nassen Böden tritt Obstbaumkrebs auf. Anfällig für Apfelwickler und Blutlaus.',
 2: 'Stark anfällig für Feuerbrand, Triebsucht, Schorf, Krebs. Auch anfällig für Stippe, Blatt- und Blutläuse.',
 3: 'Anfällig für Schorf nur in Tallagen. Mittelanfällig für Feuerbrand. Empfindlich gegenüber Schwefelmitteln.',
 4: 'Mittelstark anfällig für Feuerbrand. Widerstandsfähig gegen Schorf und Obstbaumkrebs und gegenüber vielen tierischen Schädlingen.',
 5: 'Fruchtfäulen am Baum',
 6: 'Kaum anfällig für Schorf. Sehr frosthart.',
 7: 'In geschlossenen, nassen Lagen anfällig für Schorf und Obstbaumkrebs. Mittelstark anfällig für Mehltau.',
 8: 'Stark Feuerbrand gefährdet. Auf schweren Böden tritt vermehrt Obstbaumkrebs auf. In geschlossenen Lagen Schorf möglich. Ansonsten sehr widerstandsfähig.',
 9: 'Geringe Feuerbrandanfälligkeit. In geschlossenen war

In [223]:

filter_1 = {}
for i, s in anfm.items():
    if s:
        for st in anf[i].split(","):
           if "feuer" in st.lower():
                filter_1[i] = st

In [224]:
Feuerbrand_filtered = {}
for i, s in filter_1.items():
    for st in filter_1[i].split("."):
        if "feuer" in st.lower():
            Feuerbrand_filtered[i] = st
    

In [225]:
Feuerbrand_filtered
# gibt es einen unterschied zwischen gefährdet und anfällig?
# Wie kann man das Feuerbrandgefahr einstufen?

{0: 'Feuerbrand',
 1: 'Stark Feuerbrand gefährdet',
 2: 'Stark anfällig für Feuerbrand',
 3: ' Mittelanfällig für Feuerbrand',
 4: 'Mittelstark anfällig für Feuerbrand',
 8: 'Stark Feuerbrand gefährdet',
 9: 'Geringe Feuerbrandanfälligkeit',
 13: ' Wenig feuerbrandgefährdet',
 19: ' Mittelstark anfällig für Feuerbrand',
 22: 'Stark Feuerbrandgefährdet',
 23: 'Stark Feuerbrand',
 24: 'Stark Feuerbrand gefährdet',
 25: ' Mittelstark Feuerbrand gefährdet',
 29: 'Stark feuerbrand- und mehltauanfällig',
 30: ' Widerstandsfähig gegen Feuerbrand',
 31: 'Stark Feuerbrand gefährdet',
 32: 'Stark Feuerbrand und Obstbaumkrebs gefährdet',
 35: 'Gering Feuerbrand gefährdet',
 36: 'Mittelstark Feuerbrand gefährdet',
 37: 'Mittelstark Feuerbrand gefährdet',
 41: 'Geringe Feuerbrandgefahr',
 43: 'Stark Feuerbrandgefährdet',
 44: 'Bei Temperaturschwankungen auf Feuerbrand achten',
 45: 'Feuerbrand gefährdet',
 50: 'Wenig Feuerbrand und Schorf gefährdet',
 53: 'Mittelstark Feuerbrand gefährdet',
 55: ' 